In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install cudf

In [ ]:
%%time
import sys
!cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

# Imports

In [ ]:
from glob import glob
import pandas as pd
import cudf
from tqdm.notebook import tqdm
import numpy as np
import gc

In [ ]:
all_files = glob('/kaggle/input/test-recsys/sbermarket_tab_2_*/*')

In [ ]:
%%time
dfs = []
for path in all_files[:15]:
    dfs.append(
        # reduce_mem_usage(
            cudf.read_csv(path, 
                          usecols=['user_id', 'order_id', 'line_item_id', 'price', 'quantity', 'discount',
                                   'product_id', 'master_category_id', 'parent_category_id'])
        # )
    )
total_df = cudf.concat(dfs, axis=0)
del dfs

dfs = []
for path in all_files[15:]:
    dfs.append(
        # reduce_mem_usage(
            cudf.read_csv(path, 
                          usecols=['user_id', 'order_id', 'line_item_id', 'price', 'quantity', 'discount',
                                   'product_id', 'master_category_id', 'parent_category_id'])
        # )
    )
dfs.append(total_df)
total_df = cudf.concat(dfs, axis=0)
del dfs


print('Total df shape', total_df.shape)

orders_df = cudf.read_csv('/kaggle/input/test-recsys/kaggle_tab_1345/tab_1_orders.csv')
print('Orders shape', orders_df.shape)

In [ ]:
products_df = cudf.read_csv('/kaggle/input/test-recsys/kaggle_tab_1345/tab_5_product_properties.csv')
print('Orders shape', products_df.shape)

In [ ]:
submission = cudf.read_csv('/kaggle/input/test-recsys/sample_submission.csv')
print('Submission shape', submission.shape)

In [ ]:
min_count = 10_000

full_len = len(total_df)
product_id_vc = total_df.product_id.value_counts()
top_items_by_count = product_id_vc[product_id_vc > min_count].index


total_df = total_df[total_df['product_id'].isin(top_items_by_count)]
print(
    (min_count / full_len) * 100
)
print(total_df.shape)
total_df.product_id.value_counts()

In [ ]:
total_df_orders = total_df.order_id.unique()

Remove users with only one order

In [ ]:
# DEBUG_USERS_SIZE = 10_000 * 50
# def filter_user_with_one_order(df, orders_in_total_df):
#     df = df[
#         df.order_id.isin(orders_in_total_df)
#     ]
#     orders_group_by_user_id = df.groupby('user_id')
#     orders_count_by_users = orders_group_by_user_id['order_id'].count()
#     users_with_more_than_one_order = orders_count_by_users[
#         (orders_count_by_users != 1)
#     ].index
#     # users_with_more_than_one_order = np.random.choice(users_with_more_than_one_order.to_array(), DEBUG_USERS_SIZE)
#     return df[df['user_id'].isin(users_with_more_than_one_order)]
# users_with_more_than_one_order = filter_user_with_one_order(orders_df, total_df_orders)

# users_with_more_than_one_order_pd = users_with_more_than_one_order.to_pandas()

Make dataframe with feature order ids and target ored id

In [ ]:
#dataset = pd.read_csv('/kaggle/input/sbermarketdata/dataset.csv')

In [ ]:
# %%time
# def get_featurs_target_apply(df):
#     time = df['order_created_time'].sort_values().index
    
#     target_idx = time[-1]
#     target = df.loc[target_idx]
    
#     features_idx = time[:-1]
#     features = df.loc[features_idx]
#     return (features.order_id.values, target.order_id)

# dataset = users_with_more_than_one_order_pd.groupby('user_id').apply(get_featurs_target_apply).reset_index()
# dataset = dataset.set_index('user_id')
# dataset

In [ ]:
res_df = pd.read_pickle('/kaggle/input/sbermarketdata/orders')

In [ ]:
features = res_df['features']
target = res_df['target_idx']

In [ ]:
# res_df['user_id'] = dataset.index
# res_df['features'] = features
# res_df['target_idx'] = target

In [ ]:
# import pickle

# with open('orders', 'rb') as f:
#     res_df_read = pickle.load(f)
    
# with open('orders', 'wb') as f:
#     pickle.dump(res_df, f)

From feature orders aggregate and extract information

In [ ]:
def price_agg(price):
    return (price.mean(), price.std(), price.median())

def pad_if_need(a, l):
    if len(a) != l:
        a = np.pad(a, (0, l - len(a)))
        return a
    
    return a
    

def quantity_agg(quantity):
    return (quantity.mean(), quantity.std(), quantity.median())

def extract_features(df,topk=25):
    top_parent_category = df.parent_category_id.value_counts()[:topk].index.to_array()
    top_parent_category = pad_if_need(top_parent_category, topk)
    
    top_product_id = df.product_id.value_counts()[:topk].index.to_array()    
    top_product_id = pad_if_need(top_product_id, topk)
        
    price_agg_res = price_agg(df.price)
    quantity_agg_res = quantity_agg(df.quantity)
    
    return np.concatenate((
        top_parent_category,
        top_product_id,
        price_agg_res,
        quantity_agg_res
    )).astype('float32')


def get_target_vector(order):
    return order.product_id.values

In [ ]:
# features = [i[0] for i in dataset.iloc[:,-1]]


features = res_df['features']
features = np.concatenate(features)

target = res_df['target_idx']

# target = [i[1] for i in dataset.iloc[:,-1]]


user_id = res_df.user_id

In [ ]:
DEBUG_SIZE = 100_000 * 5

In [ ]:
features_total_df = total_df[total_df.order_id.isin(features[:DEBUG_SIZE])].to_pandas()
features_total_df_gb_user_id = features_total_df.groupby('user_id')

In [ ]:
user_ids = features_total_df_gb_user_id.groups.keys()

In [ ]:
target_total_df = total_df[total_df.order_id.isin(target) & total_df.user_id.isin(user_ids)].to_pandas()
target_total_df_gb_user_id = target_total_df.groupby('user_id')

In [ ]:
def get_target_apply(df):
    return df.product_id.unique()

In [ ]:
%%time
y = target_total_df_gb_user_id.apply(get_target_apply)

In [ ]:
features_product_list = features_total_df_gb_user_id.apply(get_target_apply)

In [ ]:
pd.DataFrame(features_product_list, columns = ['lists']).to_parquet('features_product_list.parquet')

In [ ]:
# features_total_df

In [ ]:
# tmp = features_total_df.drop(['order_id','line_item_id',
#                               'price','quantity',
#                               'discount','master_category_id',
#                               'parent_category_id'], axis = 1)

In [ ]:
# tmp = tmp.drop_duplicates()
# import pickle
# with open('feat_prod.parquet','wb') as f:
#     tmp.to_parquet(f)

In [ ]:
# tmp = tmp.drop_duplicates()
# tmp = pd.get_dummies(tmp,columns = ['product_id'])

In [ ]:
# import pickle
# with open('feat_prod.pkl','wb') as f:
#     pickle.dump(tmp,f)

In [ ]:
# features_product = tmp.groupby('user_id').sum()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def extract_features_submission(df,topk=7):
    top_parent_category = df.parent_category_id.value_counts()[:topk].index
    top_parent_category = pad_if_need(top_parent_category, topk)
    
    top_product_id = df.product_id.value_counts()[:topk].index
    top_product_id = pad_if_need(top_product_id, topk)
        
    price_agg_res = price_agg(df.price)
    quantity_agg_res = quantity_agg(df.quantity)
    
    return np.concatenate((
        top_parent_category,
        top_product_id,
        price_agg_res,
        quantity_agg_res
    )).astype('float32')

In [ ]:
%%time
X = features_total_df_gb_user_id.apply(extract_features_submission)

In [ ]:
matrix = X.reset_index().merge(y.reset_index(), left_on='user_id', right_on='user_id')
matrix = matrix.rename(columns={"0_x": "features", "0_y": "target_ids"})
# matrix.to_csv('dataset', index='user_id')

In [ ]:
# matrix['features'][0].shape

In [ ]:
# import pickle
# with open('dataset_v3', 'wb') as f:
#     pickle.dump(matrix, f)

In [ ]:
# import pickle
# with open('dataset_v3', 'rb') as f:
#     matrix = pickle.load(f)

In [ ]:
matrix['features'].values.shape

In [ ]:
# with open('dataset_v2', 'wb') as f:
#     pickle.dump(matrix, f)

In [ ]:
#matrix = matrix.iloc[:10_000]
uniq_target = np.unique(
    np.hstack(matrix['target_ids'].values)
)

In [ ]:
uniq_target

In [ ]:
import pickle
with open('uniq_target.pkl', 'wb') as f:
    pickle.dump(uniq_target,f)

In [ ]:
# def concat_users_features_item_features(feats, target, all_targets):
#     x = []
#     y = []
#     for cur_target in all_targets:          
#         _x = np.hstack((
#             feats, cur_target
#         ))
        
        
#         assert _x.shape[0] == 13
#         _y = cur_target in target
        
        
#         x.append(_x)
#         y.append(_y)
        
#     return x, np.array(y).astype('int32')

# x,y = concat_users_features_item_features(matrix.features[0], matrix.target_ids[0], uniq_target)

In [ ]:
X = np.vstack(matrix.features.values)

# Product features

In [ ]:
def get_product_features(df):
    tmp = df.copy()
    tmp = tmp.drop(['user_id','order_id', 'line_item_id'],
                   axis = 1)
    tmp = tmp.groupby('product_id').agg(['mean','std','median'])
    tmp.columns = [a+'_'+b for a,b in tmp.columns]
    tmp = tmp.drop(['master_category_id_std','master_category_id_median',
                      'parent_category_id_std','parent_category_id_median'],
            axis = 1)
    tmp.columns = list(tmp.columns[:-2]) +['parent_category', 'master_category']
    tmp.reset_index(inplace = True)
    return tmp

In [ ]:
product_features = get_product_features(features_total_df)

In [ ]:
product_features = product_features[product_features.product_id.isin(uniq_target)]

In [ ]:
product_features

In [ ]:
# import pickle
# with open('prod.pkl', 'wb') as f:
#     pickle.dump(product_features, f)

# Co-Product features

In [ ]:
features_product

In [ ]:
tmp = np.concatenate(np.array(features_product))

In [ ]:
del tmp

In [ ]:
del total_df
del orders_df
del total_df_orders

In [ ]:
from tqdm import tqdm

def get_cooccurence_features(features_product, uniq_target):
    result = pd.DataFrame(features_product.copy()).reset_index()
    result = pd.DataFrame(result.values.repeat(len(uniq_target), axis = 0), columns=result.columns)
    tmp = uniq_target.copy()
    tmp = pd.DataFrame(tmp[:None])
    tmp = pd.DataFrame(np.concatenate([tmp.values]*len(features_product.index))[:None])
    result = pd.concat([result, tmp], axis = 1)
    del tmp
    result.columns = ['user_id', 'products', 'cur_product']
    result['nij'] = 0
    result['nij/ni'] = 0
    ni = {}
    all_prods = np.concatenate(np.array(features_product))
    for i in all_prods:
        if i in ni:
            ni[i]+=1
        else:
            ni[i] = 0
    del all_prods
    for i in tqdm(range(len(result.index))):
        cur_prod = result.iloc[i,2]
        nij = []
        nij_div_nj = []
        for j in result.iloc[i,1]:
            if j == cur_prod:
                continue
            nij.append(0)
            for k in range(features_product.shape[0]):
                if cur_prod in features_product.iloc[k] and j in features_product.iloc[k]:
                    nij[-1]+=1
            nij_div_nj = nij[-1]/ni[j]
        result.iloc[i,3] = np.mean(nij)
        result.iloc[i,4] = np.mean(nij_div_nj)
    return result

In [ ]:
get_cooccurence_features(features_product, uniq_target)

In [ ]:
# import pickle
# with open('user_prod.pkl', 'wb') as f:
#     pickle.dump(features_product, f)

In [ ]:
whole_X = np.repeat(X, len(uniq_target), axis=0)
whole_y = np.repeat(np.zeros_like(uniq_target), len(X), axis=0)


In [ ]:
matrix

In [ ]:
del X

In [ ]:
import gc
gc.collect()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
idxes = le.fit_transform(uniq_target)

In [ ]:
for idx, _cur in tqdm(enumerate(matrix.target_ids)):    
    start_idx = idx * len(uniq_target)
    end_idx = (idx + 1) * len(uniq_target)
    whole_X[start_idx:end_idx] = matrix['features'].iloc[idx]
    # whole_X[start_idx:end_idx][:,0] = matrix['user_id'].iloc[idx]
    target_idxes = le.transform(_cur)
    whole_y[start_idx:end_idx][target_idxes] = 1

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
idxes = le.fit_transform(uniq_target)

for idx, _cur in tqdm(enumerate(matrix.target_ids)):    
    start_idx = idx * len(uniq_target)
    end_idx = (idx + 1) * len(uniq_target)
    whole_X[start_idx:end_idx] = matrix['features'].iloc[idx]
    whole_X[start_idx:end_idx][:,-1] = np.arange(len(le.classes_))
    whole_X[start_idx:end_idx][:,-2] = matrix['user_id'].iloc[idx]
    target_idxes = le.transform(_cur)
    whole_y[start_idx:end_idx][target_idxes] = 1

matrix['user_id'].iloc[idx]

In [ ]:
del orders_df
del total_df
#del dataset
#del users_with_more_than_one_order_pd
del products_df
del dfs

In [ ]:
gc.collect()

# Co-occurence

In [ ]:
whole_X.shape

# Train model

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [ ]:
# X_train = X[:392 * 400]
# y_train = y[:392 * 400]

# X_test = X[392 * 400:]
# y_test = y[392 * 400:]

In [ ]:
# whole_y, whole_X
X_train, X_test, y_train, y_test = train_test_split(whole_X, whole_y, 
                                                    test_size=0.33, 
                                                    random_state=42)

In [ ]:
del whole_X
del whole_y
gc.collect()

In [ ]:
import catboost

In [ ]:
# train_dataset = catboost.Pool(
#     X_train, labels=y_train, weight=[0.1, 0.2, 0.3]
# )

In [ ]:
sum_neg = len(y_train) - y_train.sum()
pos_sum = y_train.sum()

In [ ]:
catboost_params = {
    'iterations':100, 
    # 'learning_rate':0.1, 
#                    'depth':10, 
#                    'l2_leaf_reg':10, 
    'random_state':7,
    'eval_metric':'AUC',
#                    'thread_count':10,  
#                    'od_type': "Iter",'od_wait':10
                  }

cbc = CatBoostClassifier(
    task_type='GPU', 
    devices='0',
    iterations=200, 
    eval_metric='AUC',
    # class_weights={0:1, 1:100},
)

cbc.fit(X_train, y_train, 
        eval_set=(X_test, y_test),
          # plot=True, 
        # class_weights={0:1, 1:100},
        verbose=True, 
          # cat_features=cat_features_names
         )

In [ ]:
cbc.save_model('catboost_submission')

In [ ]:
preds = cbc.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_score=preds, y_true=y_test)

# Submission

In [ ]:
submission = pd.read_csv('/kaggle/input/test-recsys/sample_submission.csv')


In [ ]:
submission_user_ids = submission['Id']

In [ ]:
submission_users = total_df[total_df.user_id.isin(submission_user_ids)]
submission_users = submission_users.to_pandas()

In [ ]:
submission_groupby = submission_users.groupby('user_id')

In [ ]:
submission_groupby.apply(extract_features_submission)

In [ ]:
# gb = submission_df.iloc[:100_0000].to_pandas()
# gb = gb.groupby('user_id')

In [ ]:
# %%time
# features = gb.apply(extract_features_submission)

In [ ]:
# # total_df
# submission_df = total_df[total_df.user_id.isin(user_ids)]

In [ ]:
# x,y = concat_users_features_item_features(features[0], targets[0], all_targets)

In [ ]:
# def concat_users_features_item_features_submission(feats, all_targets, model, topk=50):
#     x = []
#     y = []
#     for cur_target in all_targets:          
#         _x = np.hstack((
#             feats, cur_target
#         ))
        
        
#         assert _x.shape[0] == 13        

        
#         x.append(_x)
        
#     features = np.vstack(x)    
#     preds = model.predict_proba(features)[:,1]
#     idx = preds.argsort()[::-1][:50]    
#     return all_targets[idx]

# preds = concat_users_features_item_features_submission(features.values[0], np.array(all_targets.tolist()),
#                                                               cbc)

In [ ]:
sub_features = concat_users_features_item_features_submission(features.iloc[0], all_targets)